In [2]:
import requests
import io
from PIL import Image
import os
import time
from concurrent.futures import ThreadPoolExecutor
import re
import pandas as pd
from concurrent.futures import ThreadPoolExecutor, as_completed


In [7]:
import google.generativeai as genai

genai.configure(api_key="")
model = genai.GenerativeModel("gemini-1.5-flash")

def get_steps(answer):
    prompt = (
            "Please break down the following answer into clear, logical steps "
            "steps must  be related to company scrapUncle. "
            "steps must be capable of being used as cartoonish image prompts"
            "Use concise step titles, and aim for 4–5 steps if the answer is normal, or 1–2 steps if it's very brief. "
            "Structure the output strictly as:\n\n"
            "Step Title: <title>\nDescription: <explanation>\n\n"
            "Even if the answer does not contain any apparent steps, include a sentence that can later be used to create a prompt for image generation.\n\n"
            f"Answer:\n{answer}"
        )

    try:
        response = model.generate_content(prompt)
        lines = response.text.split("\n")
        step_titles = [
            line.replace("Step Title:", "").strip()
            for line in lines
            if line.lower().startswith("step title:")
        ]
        return step_titles
    except Exception as e:
        print("Error with Gemini step gen:", e)
        return []

def get_steps_df(df):
    return [get_steps(answer) for answer in df['answer']]



In [25]:
# ========== 1. Core Image Generation ==========

def generate_clipdrop_image(step_name, df_name, output_dir, step_index=None):
#     API_KEY = "fa85b04d973125afb4c853a19dc412c937"
    API_KEY = "22c2be801175f54c94afb768d95089"
    ENDPOINT = "https://clipdrop-api.co/text-to-image/v1"

    prompt_templates = {
        "location_df": "Cartoon-style image showing: “{step}” in the context of ScrapUncle pickup locations — customer using map or app to select address, with a uniformed ScrapUncle agent nearby.",
        "payment_methods_df": "Cartoon-style image showing: “{step}” in ScrapUncle’s payment flow — customer paying via UPI, QR code, or cash to a ScrapUncle agent in uniform.",
        "pickup_process_df": "Cartoon-style image showing: “{step}” during ScrapUncle pickup — agent collecting scrap from a doorstep, van parked, items in bags or boxes.",
        "pricing_and_rates_df": "Cartoon-style image showing: “{step}” related to ScrapUncle pricing — customer checking scrap prices on phone, rate board in background.",
        "process_and_procedures_df": "Cartoon-style image showing: “{step}” in ScrapUncle’s workflow — agent scanning, weighing scrap, showing step-by-step process.",
        "quality_df": "Cartoon-style image showing: “{step}” in ScrapUncle’s quality handling — careful sorting, weighing, with clean, safe environment.",
        "residential_services_df": "Cartoon-style image showing: “{step}” in a residential setting — ScrapUncle agent politely collecting scrap at a family home.",
        "service_areas_df": "Cartoon-style image showing: “{step}” across ScrapUncle service areas — map with pins, vans moving between localities.",
        "services_and_features_df": "Cartoon-style image showing: “{step}” as part of ScrapUncle features — highlights like fast pickup, instant payment, verified staff.",
        "technology_and_platform_df": "Cartoon-style image showing: “{step}” via ScrapUncle’s platform — app interface, notifications, customer tracking pickup.",
        "time_df": "Cartoon-style image showing: “{step}” related to pickup timing — calendar slot selection, agent arriving on time with watch or phone open.",
        "account_and_support_df": "Cartoon-style image showing: “{step}” in ScrapUncle support — customer talking to support via phone or chat, friendly support agent helping, both wearing ScrapUncle branding.",
        "business_services_df": "Cartoon-style image showing: “{step}” for ScrapUncle's business scrap pickup — large office or warehouse setting, agent handling bulk scrap, company staff observing.",
        "materials_and_acceptance_df": "Cartoon-style image showing: “{step}” in ScrapUncle material check — agent verifying scrap type (plastic, metal, paper), acceptance list displayed visibly.",
        "offers_and_promotions_df": "Cartoon-style image showing: “{step}” related to ScrapUncle offers — app showing coupons, agent giving discount slip, festive or promotional visuals included.",
        "service_availability_df": "Cartoon-style image showing: “{step}” in terms of ScrapUncle service availability — app or web showing 'Available' or 'Not available' in location, user checking it before booking.",
        "cost_and_pricing_df": "Cartoon-style image showing: “{step}” related to ScrapUncle pricing — customer checking scrap prices on phone, rate board in background."
    }
    headers = {"x-api-key": API_KEY}

    if df_name not in prompt_templates:
        print(f"❌ Error: Unknown df_name '{df_name}'")
        return

    prompt = prompt_templates[df_name].format(step=step_name)[:250]
    data = {"prompt": prompt}

    try:
        response = requests.post(ENDPOINT, headers=headers, json=data)
        if response.status_code == 200:
            image = Image.open(io.BytesIO(response.content))
            safe_step = step_name.replace(' ', '_').replace('/', '_').replace(':', '')
            filename = f"{step_index + 1:02d}_{safe_step}.png"
            save_path = os.path.join(output_dir, filename)
            image.save(save_path)
            print(f"✅ Saved: {save_path}")
        else:
            print(f"❌ Error {response.status_code}: {response.text}")
    except Exception as e:
        print(f"🔥 ClipDrop failed: {e}")

# ========== 2. Batch per DataFrame ==========

def generate_images_from_df(df, steps_list, base_output_dir, df_name, delay_seconds=2, max_workers=4):
    for q_idx, step_group in enumerate(steps_list):
        question_text = df['question'].iloc[q_idx]
        clean_question = question_text.replace('?', '').replace(':', '').replace(' ', '_')
        question_dir = os.path.join(base_output_dir, f"q_{q_idx:02d}_{clean_question}")
        os.makedirs(question_dir, exist_ok=True)

        print(f"🛠️ Generating for: {question_text}")

        with ThreadPoolExecutor(max_workers=max_workers) as executor:
            futures = []
            for step_idx, step_text in enumerate(step_group):
                futures.append(
                    executor.submit(
                        generate_clipdrop_image,
                        step_name=step_text,
                        df_name=df_name,
                        output_dir=question_dir,
                        step_index=step_idx
                    )
                )
                time.sleep(delay_seconds / 2)

            for future in as_completed(futures):
                future.result()

        print(f"✅ Done: {len(step_group)} steps for question {q_idx}\n")

# ========== 3. Orchestrate Across All DataFrames ==========

def run_full_scrapuncle_pipeline(all_dfs, output_base, delay_seconds=2):
    for df_name, df in all_dfs.items():
        print(f"\n📦 Processing: {df_name}")
        steps = get_steps_df(df)  # You should have a valid get_steps_df function
        df_output_dir = os.path.join(output_base, df_name.replace('_df', ''))
        os.makedirs(df_output_dir, exist_ok=True)

        generate_images_from_df(
            df,
            steps,
            base_output_dir=df_output_dir,
            df_name=df_name,
            delay_seconds=delay_seconds
        )

# ========== 4. Prepare Folders and Filtered DataFrames ==========

def prepare_scrapuncle_folders_and_dataframes(df, output_base):

    def canonical_name(text: str) -> str:
        """
        Turn arbitrary category text into a predictable snake-case slug.
        Rules:
          • lower-case
          • '&' → 'and'
          • keep letters/digits, everything else → '_'
          • collapse multiple underscores
          • strip leading/trailing underscores
        """
        text = text.lower().replace('&', 'and')
        text = re.sub(r'[^a-z0-9]+', '_', text)          # non-alphanum → _
        text = re.sub(r'_+', '_', text)                  # collapse ___ → _
        return text.strip('_')

    df_variable_names, all_dfs = [], {}
    os.makedirs(output_base, exist_ok=True)

    for cat in df['category'].unique():
        slug = canonical_name(cat)
        full_var_name = f"{slug}_df"

        df_filtered = df[df['category'] == cat].reset_index(drop=True)
        df_variable_names.append(full_var_name)
        all_dfs[full_var_name] = df_filtered

        os.makedirs(os.path.join(output_base, slug), exist_ok=True)

    return df_variable_names, all_dfs


In [9]:
def generate(df):
    output_base = "output_images"


    # Prepare folders + split into category-specific dataframes
    df_names, all_dfs = prepare_scrapuncle_folders_and_dataframes(df, output_base)

    # Run full image generation pipeline
    run_full_scrapuncle_pipeline(all_dfs, output_base)


In [13]:
df = pd.read_excel(r"C:\Users\tishya\OneDrive\Desktop\new questions.xlsx")
sample_df = df.sample(3)
sample_df

,id,question,answer,category,confidence,source_url,key_term,question_type,page_title,answer_length,is_variation
17,18,Do you pay different rates for sorted scrap?,"Yes, sorted scrap items often fetch better rat...",Pickup Process,high,NaN,pickup,how,ScrapUncle - Recycling Made Easy,81,NaN
0,1,Do you offer services in gated communities or ...,"Yes, we provide pickup services in apartments ...",Account & Support,medium,NaN,safety,is,ScrapUncle - Recycling Made Easy,87,NaN
9,10,Can I track my pickup agent live?,"Currently, live agent tracking isn’t supported...",Materials & Acceptance,high,NaN,cancel,how,ScrapUncle - Recycling Made Easy,110,NaN


In [14]:
generate(sample_df)


📦 Processing: pickup_process_df
🛠️ Generating for: Do you pay different rates for sorted scrap?
✅ Saved: output_images\pickup_process\q_00_Do_you_pay_different_rates_for_sorted_scrap\01_Sorting_Triumph.png
✅ Saved: output_images\pickup_process\q_00_Do_you_pay_different_rates_for_sorted_scrap\02_Higher_Price_Tag.png
✅ Done: 2 steps for question 0


📦 Processing: account_and_support_df
🛠️ Generating for: Do you offer services in gated communities or apartments?
✅ Saved: output_images\account_and_support\q_00_Do_you_offer_services_in_gated_communities_or_apartments\01_Access_Granted!.png
✅ Saved: output_images\account_and_support\q_00_Do_you_offer_services_in_gated_communities_or_apartments\02_Apartment_Arrival!.png
✅ Done: 2 steps for question 0


📦 Processing: materials_and_acceptance_df
🛠️ Generating for: Can I track my pickup agent live?
✅ Saved: output_images\materials_and_acceptance\q_00_Can_I_track_my_pickup_agent_live\01_No_Live_Agent_Tracking.png
✅ Saved: output_images\materials

In [22]:
df = pd.read_csv(r"C:\Users\tishya\SELF MADE\SCRAP_UNCLE\missing_questions_by_category.csv")

In [23]:
df = df.iloc[15:]
df

,id,question,answer,category,confidence,source_url,key_term,question_type,page_title,answer_length,is_variation
15,75,HOW TO DOWNLOAD SCRAPUNCLE APP,Download the ScrapUncle App from the app store...,Pricing & Rates,high,https://scrapuncle.com/,download,what,ScrapUncle - Recycling Made Easy,71,False
16,78,WHICH SERVICE GREAT IS BEST FOR ME,Compare service options and select what suits ...,Pricing & Rates,medium,https://scrapuncle.com/,great,which,ScrapUncle - Recycling Made Easy,72,False
17,82,I'M CURIOUS ABOUT: WHICH SCRAP RATES IS BEST F...,Use the ScrapUncle app to compare current scra...,Pricing & Rates,medium,https://scrapuncle.com/,scrap rates,which,ScrapUncle - Recycling Made Easy,77,True
18,119,HOW DOES WEIGHING WORK,Scrap is weighed using portable digital machin...,Quality,very_high,https://scrapuncle.com/home,NaN,ScrapUncle - Recycling Made Easy,NaN,94,False
19,124,COULD YOU PLEASE EXPLAIN: HOW TO USE SCHEDULE ...,You use schedule pickup by selecting a time sl...,Time,medium,https://scrapuncle.com/,schedule pickup,how,ScrapUncle - Recycling Made Easy,107,True


In [26]:
generate(df)


📦 Processing: pricing_and_rates_df
🛠️ Generating for: HOW TO DOWNLOAD SCRAPUNCLE APP
✅ Saved: output_images\pricing_and_rates\q_00_HOW_TO_DOWNLOAD_SCRAPUNCLE_APP\01_App_Store_Download.png
✅ Saved: output_images\pricing_and_rates\q_00_HOW_TO_DOWNLOAD_SCRAPUNCLE_APP\02_App_Launch.png
✅ Done: 2 steps for question 0

🛠️ Generating for: WHICH SERVICE GREAT IS BEST FOR ME
✅ Saved: output_images\pricing_and_rates\q_01_WHICH_SERVICE_GREAT_IS_BEST_FOR_ME\01_Assessing_Recycling_Needs.png
✅ Saved: output_images\pricing_and_rates\q_01_WHICH_SERVICE_GREAT_IS_BEST_FOR_ME\02_Comparing_Service_Options.png
✅ Done: 2 steps for question 1

🛠️ Generating for: I'M CURIOUS ABOUT: WHICH SCRAP RATES IS BEST FOR ME
✅ Saved: output_images\pricing_and_rates\q_02_I'M_CURIOUS_ABOUT_WHICH_SCRAP_RATES_IS_BEST_FOR_ME\01_Download_&_Open_ScrapUncle.png
✅ Saved: output_images\pricing_and_rates\q_02_I'M_CURIOUS_ABOUT_WHICH_SCRAP_RATES_IS_BEST_FOR_ME\02_Input_Scrap_Details.png
✅ Saved: output_images\pricing_and_rates\q_0